In [10]:
import pandas as pd
import re
import ast
import numpy as np

df = pd.read_csv("../ocr/output/ocr_results.csv", encoding='latin1')

In [11]:
def parse_text(text):
    if not isinstance(text, str):
        return []

    title_re = re.compile(r'(AMENDMENT TO ORDINANCE|MOTION|RESOLUTION) ?-? ?(NO\.)? ?([A-Z0-9\-]+\,?[A-Z0-9\-]?) - BY: (.*)')
    action_re = re.compile(r'(ACTION:\n)(Amendment|As Amended|Adopt|Enter Executive Session)')
    brief_re = re.compile(r'Brief:\n(.*)(?=Annotation:)', re.DOTALL)
    annotation_re = re.compile(r'Annotation:\n(.*)')
    moved_by_re = re.compile(r'MOVED BY:\n(.*)')
    seconded_by_re = re.compile(r'SECONDED BY:\n(.*)')
    votes_re = re.compile(r'(.*?)(?: - \d+)?\n(YEAS|NAYS|ABSTAIN|ABSENT|RECUSED):')
    motion_passed_re = re.compile(r'AND THE MOTION (PASSED|FAILED).')
    withdrawn_re = re.compile(r'WITHDRAWN.')

    motion_blocks = re.split(r'(AND THE MOTION (PASSED|FAILED).\n|WITHDRAWN.\n)', text)

    parsed_list = []
    
    for block in motion_blocks:
        if not isinstance(block, str):
            continue
            
        matches = list(title_re.finditer(block))
        for i, title_match in enumerate(matches):
            parsed = {"motionDetails": {}}

            parsed["motionDetails"]["title"] = title_match.group(1) + " " + title_match.group(3)
            parsed["motionDetails"]["proposedBy"] = title_match.group(4)

            action_match = action_re.search(block, pos=title_match.end())
            if action_match:
                parsed["motionDetails"]["action"] = action_match.group(2)

            brief_match = brief_re.search(block, pos=title_match.end())
            if brief_match:
                parsed["motionDetails"]["brief"] = brief_match.group(1).strip()

            annotation_match = annotation_re.search(block, pos=title_match.end())
            if annotation_match:
                parsed["motionDetails"]["annotation"] = annotation_match.group(1)

            moved_by_match = moved_by_re.search(block, pos=title_match.end())
            if moved_by_match:
                parsed["motionDetails"]["movedBy"] = moved_by_match.group(1)

            seconded_by_match = seconded_by_re.search(block, pos=(moved_by_match.end() if moved_by_match else 0))
            if seconded_by_match:
                parsed["motionDetails"]["secondedBy"] = seconded_by_match.group(1)

            parsed["motionDetails"]["votingDetails"] = {}

            for votes_match in votes_re.finditer(block, pos=(seconded_by_match.end() if seconded_by_match else 0)):
                vote_type = votes_match.group(2).lower()
                members = votes_match.group(1).split(",")
                for member in members:
                    parsed["motionDetails"]["votingDetails"][member.strip()] = vote_type

            motion_passed_match = motion_passed_re.search(block, pos=(seconded_by_match.end() if seconded_by_match else 0))
            if motion_passed_match:
                parsed["motionDetails"]["motionPassed"] = True if motion_passed_match.group(1) == "PASSED" else False

            withdrawn_match = withdrawn_re.search(block, pos=(seconded_by_match.end() if seconded_by_match else 0))
            if withdrawn_match:
                parsed["motionDetails"]["withdrawn"] = True

            if parsed["motionDetails"]["votingDetails"]:
                parsed_list.append(parsed)

    return parsed_list

dfa = df.copy()
dfa['parsed_text'] = dfa['text'].apply(parse_text)

dfa = dfa.explode('parsed_text').reset_index(drop=True)
dfa

,filename,chunk,text,parsed_text
0,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""6/23/23, 5:41 PM\ncityofno.granicus.com/Min...",NaN
1,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...",{'motionDetails': {'title': 'RESOLUTION R-23-1...
2,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...",{'motionDetails': {'title': 'RESOLUTION R-23-1...
3,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...",{'motionDetails': {'title': 'RESOLUTION R-23-1...
4,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...","{'motionDetails': {'title': 'MOTION M-23-152',..."
...,...,...,...,...
282,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,3,"""""""eadline 9/22/23).\nCONTINUED TO THE MEETING...",{'motionDetails': {'title': 'RESOLUTION R-23-2...
283,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,3,"""""""eadline 9/22/23).\nCONTINUED TO THE MEETING...","{'motionDetails': {'title': 'MOTION M-23-266',..."
284,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,3,"""""""eadline 9/22/23).\nCONTINUED TO THE MEETING...","{'motionDetails': {'title': 'MOTION M-23-273',..."
285,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,3,"""""""eadline 9/22/23).\nCONTINUED TO THE MEETING...","{'motionDetails': {'title': 'MOTION M-23-275',..."


In [12]:
def parse_text(text):
    if not isinstance(text, str):
        return []

    text = re.sub(r'https://cityofno\.granicus\.com/MinutesViewer[^\s\n]*', '', text, flags=re.MULTILINE)

    title_re = re.compile(r'(CAL\. NO\.) ?-? ?([A-Z0-9,\- ]+)(?:- BY: (.*?))(?: \(BYREQUEST\))?(?=Brief:|\\n|$)', re.DOTALL)
    action_re = re.compile(r'(ACTION:\n)(Amendment|As Amended|Adopt|Enter Executive Session)')
    brief_re = re.compile(r'Brief:\n(.*?)(?=Annotation:)', re.DOTALL)
    annotation_re = re.compile(r'Annotation:\n(.*)')
    moved_by_re = re.compile(r'MOVED BY:\n(.*)')
    seconded_by_re = re.compile(r'SECONDED BY:\n(.*)')
    votes_re = re.compile(r'(([\w\s,]+)(?: - \d+)?)?\n(YEAS|NAYS|ABSTAIN|ABSENT|RECUSED):')
    motion_passed_re = re.compile(r'AND THE MOTION (PASSED|FAILED).')
    withdrawn_re = re.compile(r'WITHDRAWN.')

    motion_blocks = re.split(r'(AND THE MOTION (PASSED|FAILED).\n|WITHDRAWN.\n)', text)

    parsed_list = []
    
    for block in motion_blocks:
        if not isinstance(block, str):
            continue
            
        matches = list(title_re.finditer(block))
        for i, title_match in enumerate(matches):
            parsed = {"motionDetails": {}}

            parsed["motionDetails"]["title"] = title_match.group(1) + " " + title_match.group(2)
            proposedBy = title_match.group(3)
            if proposedBy:
                proposedBy = proposedBy.replace("\n", " ").replace("(BYREQUEST)", "").strip()
            parsed["motionDetails"]["proposedBy"] = proposedBy

            action_match = action_re.search(block, pos=title_match.end())
            if action_match:
                parsed["motionDetails"]["action"] = action_match.group(2)

            brief_match = brief_re.search(block, pos=title_match.end())
            if brief_match:
                parsed["motionDetails"]["brief"] = brief_match.group(1).strip()

            annotation_match = annotation_re.search(block, pos=title_match.end())
            if annotation_match:
                parsed["motionDetails"]["annotation"] = annotation_match.group(1)

            moved_by_match = moved_by_re.search(block, pos=title_match.end())
            if moved_by_match:
                parsed["motionDetails"]["movedBy"] = moved_by_match.group(1)

            seconded_by_match = seconded_by_re.search(block, pos=(moved_by_match.end() if moved_by_match else 0))
            if seconded_by_match:
                parsed["motionDetails"]["secondedBy"] = seconded_by_match.group(1)

            parsed["motionDetails"]["votingDetails"] = {}

            for votes_match in votes_re.finditer(block, pos=(seconded_by_match.end() if seconded_by_match else 0)):
                vote_type = votes_match.group(3).lower()
                members = votes_match.group(2).split(", ") if votes_match.group(2) else ["0"]
                for member in members:
                    member = member.strip()
                    if member and member != "0":
                        member = member.replace("\n", "") 
                        parsed["motionDetails"]["votingDetails"][member] = vote_type

            motion_passed_match = motion_passed_re.search(block, pos=(seconded_by_match.end() if seconded_by_match else 0))
            if motion_passed_match:
                parsed["motionDetails"]["motionPassed"] = True if motion_passed_match.group(1) == "PASSED" else False

            withdrawn_match = withdrawn_re.search(block, pos=(seconded_by_match.end() if seconded_by_match else 0))
            if withdrawn_match:
                parsed["motionDetails"]["withdrawn"] = True

            if parsed["motionDetails"]["votingDetails"]:
                parsed_list.append(parsed)

    return parsed_list

dfb = df.copy()
dfb['parsed_text'] = dfb['text'].apply(parse_text)

dfb = dfb.explode('parsed_text').reset_index(drop=True)
dfb

,filename,chunk,text,parsed_text
0,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""6/23/23, 5:41 PM\ncityofno.granicus.com/Min...",NaN
1,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...","{'motionDetails': {'title': 'CAL. NO. 34,125 '..."
2,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...","{'motionDetails': {'title': 'CAL. NO. 33,893 '..."
3,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...","{'motionDetails': {'title': 'CAL. NO. 34,017 '..."
4,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...","{'motionDetails': {'title': 'CAL. NO. 34,046 '..."
...,...,...,...,...
490,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,248 '..."
491,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,249 '..."
492,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,250 '..."
493,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,251 '..."


In [13]:
dfc = pd.concat([dfa,dfb])

dfc

,filename,chunk,text,parsed_text
0,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""6/23/23, 5:41 PM\ncityofno.granicus.com/Min...",NaN
1,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...",{'motionDetails': {'title': 'RESOLUTION R-23-1...
2,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...",{'motionDetails': {'title': 'RESOLUTION R-23-1...
3,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...",{'motionDetails': {'title': 'RESOLUTION R-23-1...
4,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,1,"""""""cityofno.granicus.com/MinutesViewer.php?vie...","{'motionDetails': {'title': 'MOTION M-23-152',..."
...,...,...,...,...
490,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,248 '..."
491,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,249 '..."
492,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,250 '..."
493,/mnt/c/Users/ayyubi/Desktop/tgi/the-great-inqu...,4,"""""""econd Street, Fourth Street, Rousseau\nStre...","{'motionDetails': {'title': 'CAL. NO. 34,251 '..."


In [14]:
votes = ['yeas', 'nays', 'recused', 'abstain', 'absent']

def dict_to_df(row):
    if pd.isnull(row):
        return pd.DataFrame({
            'title': [np.nan],
            'proposedBy': [np.nan],
            'action': [np.nan],
            'brief': [np.nan],
            'annotation': [np.nan],
            'movedBy': [np.nan],
            'council_member': [None],
            'vote': ['Not available']
        })
    
    if isinstance(row, str):
        dict_data = ast.literal_eval(row)
    else:
        dict_data = row
    
    motion_data = dict_data['motionDetails'].copy()
    voting_details = motion_data.pop('votingDetails', {})
    motion_df = pd.json_normalize(motion_data)

    voting_data = []
    for member, vote in voting_details.items():
        if vote.lower() in votes:  
            voting_data.append({'council_member': member, 'vote': vote})
    
    if not voting_data:
        voting_data.append({'council_member': None, 'vote': 'Not available'})
    
    voting_df = pd.DataFrame(voting_data)
    
    df = pd.concat([motion_df]*len(voting_df), ignore_index=True)
    df = pd.concat([df, voting_df], axis=1)
    
    return df

data_df = pd.concat(dfc['parsed_text'].apply(dict_to_df).tolist(), ignore_index=True)

data_df

,title,proposedBy,action,brief,annotation,movedBy,council_member,vote,secondedBy
0,NaN,NaN,NaN,NaN,NaN,NaN,None,Not available,NaN
1,RESOLUTION R-23-166,"COUNCILMEMBERS GREEN, MORRELL,",Adopt,Encouraging the State Legislature to authorize...,(Council Rule 34. Postponement Deadline 8/18/23).,Harris,Giarrusso,yeas,ACTION:
2,RESOLUTION R-23-166,"COUNCILMEMBERS GREEN, MORRELL,",Adopt,Encouraging the State Legislature to authorize...,(Council Rule 34. Postponement Deadline 8/18/23).,Harris,Green,yeas,ACTION:
3,RESOLUTION R-23-166,"COUNCILMEMBERS GREEN, MORRELL,",Adopt,Encouraging the State Legislature to authorize...,(Council Rule 34. Postponement Deadline 8/18/23).,Harris,Harris,yeas,ACTION:
4,RESOLUTION R-23-166,"COUNCILMEMBERS GREEN, MORRELL,",Adopt,Encouraging the State Legislature to authorize...,(Council Rule 34. Postponement Deadline 8/18/23).,Harris,Moreno,yeas,ACTION:
...,...,...,...,...,...,...,...,...,...
5848,"CAL. NO. 34,252","COUNCILMEMBERSGIARRUSSO, MORENO, GREEN AND THOMAS",Adopt,An Ordinance to amend Ordinance No. 29266 M.C....,(Council Rule 34. Postponement Deadline 10/6/23).,NaN,Harris,yeas,Harris
5849,"CAL. NO. 34,252","COUNCILMEMBERSGIARRUSSO, MORENO, GREEN AND THOMAS",Adopt,An Ordinance to amend Ordinance No. 29266 M.C....,(Council Rule 34. Postponement Deadline 10/6/23).,NaN,King,yeas,Harris
5850,"CAL. NO. 34,252","COUNCILMEMBERSGIARRUSSO, MORENO, GREEN AND THOMAS",Adopt,An Ordinance to amend Ordinance No. 29266 M.C....,(Council Rule 34. Postponement Deadline 10/6/23).,NaN,Moreno,yeas,Harris
5851,"CAL. NO. 34,252","COUNCILMEMBERSGIARRUSSO, MORENO, GREEN AND THOMAS",Adopt,An Ordinance to amend Ordinance No. 29266 M.C....,(Council Rule 34. Postponement Deadline 10/6/23).,NaN,Morrell,yeas,Harris


In [ ]:
data_df.to_csv("parsed_minutes.csv", index=False)